In [284]:
##import processed push data

#/Users/joekaojoekao/PycharmProjects/push/github/visualized/select_pushes1000_1.txt
with open('/Users/joekaojoekao/PycharmProjects/push/github/visualized/select_pushes1000_1.txt', 'rb') as fin:
    s = fin.read().split('\n')
    lines = []
    for line in s:
        lines.append(line.split('\t'))
    fin.close()

sample_dict = {}
for i in lines:
    
    #print len(i)
    #if len(i) >= 3:
    if len(i) != 2:
        continue
    uid = i[0]
    push = i[1]
    if uid not in sample_dict:
        sample_dict[uid] = [push]
    else:
        sample_dict[uid].append(push)
#print sample_dict


##segment pushes

import jieba
import jieba.analyse
jieba.set_dictionary('dict.txt.big')
jieba.analyse.set_stop_words('stopword_pool/merged_stopword.txt')

sample_dict_jieba = {}
for uid in sample_dict.keys():
    pushes = sample_dict[uid]
    if len(pushes) > 0:
        sample_dict_jieba[uid] = []
        for push in pushes:
            seg_list = list(jieba.cut(push, cut_all=False))
            sample_dict_jieba[uid].append((push,seg_list))
#從ptt push開始處理
#我要uid:push&trunked word
#幹 要現切 因為產生的東西只剩uid+push(無順序無法查)

import numpy as np
import json
#set gt_numword (meaningful), gt_numpush (active)
NUM_WORD_GT = 4
NUM_PUSH_GT = 3 #for 6 fold-cv

new_pushes = {}
for k in sample_dict.keys():
    gt_push = [x for x in sample_dict[k] if len(x) > NUM_WORD_GT]
    if len(gt_push) > 0:
        new_pushes[k] = gt_push

user_list = []
for k in new_pushes.keys():
    if len(new_pushes[k]) >= NUM_PUSH_GT:
        user_list.append(k)
        
print len(sample_dict)
print len(new_pushes)
print len(user_list)
print user_list

#import nltk
from sklearn import cross_validation
#training_set = nltk.classify.apply_features(extract_features, documents)
#dict_user = {} #training
#dict_user_ans = {} #testing
CV_TIMES = 3

#list_user = [{}] * 3
#list_user_ans = [{}] * 3

list_user = [{} for i in range(3)] 
list_user_ans = [{} for i in range(3)] 


for uid in user_list:
    training_set = sample_dict_jieba[uid]
    #cv = cross_validation.KFold(len(training_set), n_folds=3, indices=True, shuffle=False, random_state=None, k=None)
    #trainset = [0,1,2,3,4,5,6,7,8,9]
    cv = cross_validation.ShuffleSplit(len(training_set), n_iter=CV_TIMES, test_size=0.33) # 2/3 for train, 1/3 for test, do 3 times  
    
    #for i in training_set:
        #print i[0]
    
    for n, (traincv, testcv) in enumerate(cv): #每個user都有3組cv的推文
        #print n, traincv, testcv
        
        train_list = [training_set[i] for i in traincv]
        test_list = [training_set[i] for i in testcv]
        
#         for i in zip(*train_list)[0]:
#             print i
#         print '---'
#         for i in zip(*test_list)[0]:
#             print i
        
        list_user[n][uid] = train_list
        list_user_ans[n][uid] = test_list
        
#         list_user[n][uid] = []
#         for i in traincv:
#             list_user[n][uid].append(training_set[i]) #list_user[第n次cv][user uid]
        
#         list_user_ans[n][uid] = []
#         for i in testcv:
#             list_user_ans[n][uid].append(training_set[i])
        
        #print uid
        #for v in list_user[0][uid]:
            #print v[0]
            #for vv in v:
                #print vv[0]


# for p in list_user[0].values()[1]:
#     for pp in p:
#         print p[0]
# print '-----------'
# for p in list_user[1].values()[1]:
#     for pp in p:
#         print p[0]

# real do cv 
final_list = []

for i in xrange(CV_TIMES):
    dict_user = list_user[i]
    dict_user_count = pttfunc.count_dict(dict_user) #count the freq
    
    dict_user_ans = list_user_ans[i]
    dict_user_count_ans = pttfunc.count_dict(dict_user_ans) #count the freq for ans(test)
    from collections import Counter
    count_all = Counter()
    for v in dict_user_count.values():
        count_all += Counter(v)

    from collections import OrderedDict
    dict_all_count = OrderedDict(sorted(dict(count_all).items(), key=lambda t: t[1], reverse=True))

    ##gen sw
    word = dict_all_count.keys()
    W_PERCENT = 0.025 #0.025
    ## setting for stopword & rareword percentage
    stopwords = [x for x in word if dict_all_count[x] >= dict_all_count[word[int(len(word) * W_PERCENT)]]]
    rarewords = [x for x in word if dict_all_count[x] <= dict_all_count[word[int(len(word) * (1-W_PERCENT))]]] #0.975

    ## stop word list
    sw_list = [x for x in stopwords] 
    ## rare word list
    rw_list = [x for x in rarewords] 
    print 'total word', len(word)
    print 'stop word', len(sw_list)
    print 'rare word', len(rw_list)


    sw_count_dict = {}
    for w in sw_list:
        sw_count_dict.setdefault(len(w),[]).append(w)


    K_NUMWORD = 6
    selected_sw = []
    for k in sw_count_dict.keys()[0:K_NUMWORD]:
        print 'num_word:',k,'\t',len(sw_count_dict[k])
        selected_sw += sw_count_dict[k]
    print len(selected_sw)
    
    
    
    ##gen vec
    import itertools
    general_vec = {}
    for uid in user_list: # for each user id
        user_len = sum([len(x) for x in dict_user_count[uid]]) #total word freq
        #print sum(v.values())
        if user_len > 0:
            #vec = [jvc_grams_count[idx].get(w, 0) for w in new_sw_list] #stopword without function words

            vec = [dict_user_count[uid].get(w, 0) for w in selected_sw]
            g_vec = [float(x) / user_len for x in vec]
            general_vec[uid] = g_vec


    general_vec_ans = {}
    for uid in user_list: # for each user id
        user_len = sum([len(x) for x in dict_user_count_ans[uid]]) #total word freq
        #print sum(v.values())
        if user_len > 0:
            #vec = [jvc_grams_count[idx].get(w, 0) for w in new_sw_list] #stopword without function words

            vec = [dict_user_count_ans[uid].get(w, 0) for w in selected_sw]
            g_vec = [float(x) / user_len for x in vec]
            general_vec_ans[uid] = g_vec

    print len(general_vec[user_list[0]])
    print len(general_vec_ans[user_list[0]])
    
    import pttfunc
    #comb = gb.getCombination(len(user_list))

    ##calculate similarity for each user pair

    sim_list = np.array((0.0, 0.0, 0.0))

    # for i, j in comb:
    #     wj_sw = pttfunc.weighted_jaccard(general_vec[user_list[i]], general_vec_ans[user_list[j]])
    #     #sim_list = np.append(sim_list, np.array((k1, k2, wj_sw), dtype=mtype))
    #     sim_list = np.vstack((sim_list, np.array((round(float(i),1), round(float(j),1), wj_sw))))

    for i in xrange(len(user_list)):
        for j in xrange(len(user_list)):
            wj_sw = pttfunc.weighted_jaccard(general_vec[user_list[i]], general_vec_ans[user_list[j]])
            sim_list = np.vstack((sim_list, np.array((round(float(i),1), round(float(j),1), wj_sw))))



    sim_list = sim_list[1:]
    sim_list = sim_list[sim_list[:,2].argsort()]
    sim_list = sim_list[::-1]



    user_sim_list = []
    for i, j, sim in sim_list:
        user_sim_list.append((user_list[int(i)], user_list[int(j)] + 'ANS', sim))
    
    final_list.append(user_sim_list)


Building prefix dict from /Users/joekaojoekao/PycharmProjects/push/github/dict.txt.big ...
DEBUG:jieba:Building prefix dict from /Users/joekaojoekao/PycharmProjects/push/github/dict.txt.big ...
Loading model from cache /var/folders/qn/v8s1xx6d7qgclyhkk1t701zc0000gn/T/jieba.u28e5fd167ca6b25e789769ad04c48668.cache
DEBUG:jieba:Loading model from cache /var/folders/qn/v8s1xx6d7qgclyhkk1t701zc0000gn/T/jieba.u28e5fd167ca6b25e789769ad04c48668.cache
Loading model cost 1.89216423035 seconds.
DEBUG:jieba:Loading model cost 1.89216423035 seconds.
Prefix dict has been built succesfully.
DEBUG:jieba:Prefix dict has been built succesfully.


999
999
718
['heavenkghs', 'nysky', 'quiet113', 'john2557', 'duo0518', 'tienhun', 'joyjack', 'vm9487', 'aftersilence', 'lingon', 'toshizo', 'yuminlin', 'fhuocrkt', 'theropod', 'kkchen', 'emptie', 'h3178378', 'zxc12385', 'Amelie27', 'buddygirl', 'chiang017', 'fromwilda', 'carters', 'Magicwind', 'glory5566', 'feather9298', 'fjdkqp', 'cojinyu', 'gayya152535', 'hnrywang', 'bugya', 'JeanSijhih', 'joyjcc', 'jeffl0402', 'Valerie06', 'SpadeR', 'cloud0528', 'xup654vu06', 'kindless', 'mylife001', 'minnie', 'ronanhuang', 'tako0988', 'treiss', 'asami', 'mineko', 'dayoa', 'akaihuang', 'sigh0602', 'wolfking623', 'koreapig5566', 'lachesis1980', 'handsomecat', 'JCS15', 'caffeine34ko', 'pprisa', 'Nick7777', 'Voldemar', 'samuelcdf', 'jaguars33', 'chx64', 'otom', 'hiro1221', 'hyde7015', 'wxtab019', 'deleteBB', 'WhyNoSmoke', 'lovemelynn', 'straight0711', 'allen5339', 'q152134', 'Sugiros', 'ursula141885', 'Monchestnut', 'captainlee', 'hoka777', 'chind', 'kaorucyc', 'surot', 'TheMiserable', 'motorolla', 'aa

In [283]:
from IPython.html.widgets import interact
def evaluate(thres_sim, cv_index):
    
    user_sim_list = final_list[cv_index]
    ans_pair_lt_thres = 0 #same user & LT
    user_pair_lt_thres = 0 # LT
    num_user = len(user_list)
    
    
    sim_list = zip(*user_sim_list)[2]
    sim_list = [value for value in sim_list if not math.isnan(value)]
    max_thres = max(sim_list)
    print max_thres
    print thres_sim
    if thres_sim > max_thres:
        return
    
    for i,j,sim in user_sim_list:
        if i in j and sim >= thres_sim:
            ans_pair_lt_thres += 1
        if sim >= thres_sim:
            user_pair_lt_thres += 1
        
    #b = len(user_list)
    precision = float(ans_pair_lt_thres) / user_pair_lt_thres
    recall = float(ans_pair_lt_thres) / num_user
    print 'my_precision:', precision #how many user their ACC are larger than Thres
    print 'my_recall:', recall
    if precision == 0 and recall == 0:
        return
    print 'f-1 measure:', 2 * (precision * recall) / (precision + recall)




interact(evaluate, thres_sim=(0.0,1.0,0.01), cv_index=(0, CV_TIMES-1)); #101

1.0
0.29
my_precision: 0.0709861838971
my_recall: 0.207520891365
f-1 measure: 0.10578629748


In [285]:
interact(evaluate, thres_sim=(0.0,1.0,0.01), cv_index=(0, CV_TIMES-1)); #

1.0
0.8
my_precision: 0.4
my_recall: 0.00278551532033
f-1 measure: 0.00553250345781


In [286]:
path = '/Users/joekaojoekao/PycharmProjects/push/github/result/'
prefix = '1000_1'
for i in xrange(CV_TIMES):
    fout = open(path+prefix+'_cv_'+str(i)+'.txt', 'wb')
    user_sim_list = final_list[i]
    for i,j,sim in user_sim_list:
        fout.write(str(i)+','+str(j)+','+str(sim)+'\n')
    fout.close()
    

In [276]:
print user_list
print len(final_list[0])
# for i, j, sim in user_sim_list:
#     if i,j,sim
user_sim_list = final_list[1]
#print user_sim_list
a = zip(*user_sim_list)[2]
print a

['akaihuang', 'jennywalk', 'C4891', 'morgan168', 'AAAB', 'AAAD', 'fst1985', 'aoiaoi', 'andy80209', 'fhuocrkt', 'wxtab019', 'keithking', 'wilsonno1', 'douglasyeh', 'Meow0129', 'chind', 'motorolla', 'Yakyuboy51', 'feather9298']
361
(nan, nan, 0.6901408450704225, 0.66666666666666663, 0.66666666666666663, 0.63157894736842102, 0.61111111111111105, 0.57142857142857151, 0.55944477972238982, 0.55174534830078426, 0.53546375681995317, 0.48845686512758191, 0.44829325710442236, 0.44177126917712689, 0.44055068836045042, 0.42707444864140376, 0.42548596112311021, 0.42352941176470593, 0.42227378190255205, 0.42191142191142189, 0.42120503396814796, 0.41607898448519043, 0.40935672514619881, 0.40772128060263652, 0.40271493212669679, 0.39919354838709681, 0.39466019417475723, 0.39160839160839161, 0.39006024096385539, 0.38819617622610131, 0.38317757009345793, 0.38129496402877705, 0.36966126656848308, 0.36641221374045796, 0.36268786486150795, 0.35506929248723568, 0.35077288941736023, 0.34768541562966659, 0.34

In [58]:
trainset = [0,1,2,3,4,5,6,7,8,9]
ss = cross_validation.ShuffleSplit(len(trainset), n_iter=3, test_size=0.33)
for traincv, testcv in ss:
        print traincv, testcv
        for i in traincv:
            print 'train', i
        for i in testcv:
            print 'test', i

[1 6 4 9 8 7] [3 0 2 5]
train 1
train 6
train 4
train 9
train 8
train 7
test 3
test 0
test 2
test 5
[8 7 1 5 4 9] [3 6 2 0]
train 8
train 7
train 1
train 5
train 4
train 9
test 3
test 6
test 2
test 0
[5 2 3 4 8 9] [6 7 1 0]
train 5
train 2
train 3
train 4
train 8
train 9
test 6
test 7
test 1
test 0


In [227]:
#########test run

## import processed push data
with open('/Users/joekaojoekao/PycharmProjects/push/github/visualized/select_pushes101.txt', 'rb') as fin:
    s = fin.read().split('\n')
    lines = []
    for line in s:
        lines.append(line.split('\t'))
    fin.close()

## make a dict
sample_dict = {}
for i in lines:
    if len(i) != 2:
        continue
    uid = i[0]
    push = i[1]
    if uid not in sample_dict:
        sample_dict[uid] = [push]
    else:
        sample_dict[uid].append(push)

## new dict with values containing push & seg_list
import jieba
import jieba.analyse
jieba.set_dictionary('dict.txt.big')
jieba.analyse.set_stop_words('stopword_pool/merged_stopword.txt')
sample_dict_jieba = {}
for uid in sample_dict.keys():
    pushes = sample_dict[uid]
    if len(pushes) > 0:
        sample_dict_jieba[uid] = []
        for push in pushes:
            seg_list = list(jieba.cut(push, cut_all=False))
            sample_dict_jieba[uid].append((push,seg_list))



## filter pushes by setting gt_numword (meaningful), gt_numpush (active)
NUM_WORD_GT = 4
NUM_PUSH_GT = 3 #for 6 fold-cv

new_pushes = {}
for k in sample_dict.keys():
    gt_push = [x for x in sample_dict[k] if len(x) > NUM_WORD_GT]
    if len(gt_push) > 0:
        new_pushes[k] = gt_push

user_list = []
for k in new_pushes.keys():
    if len(new_pushes[k]) >= NUM_PUSH_GT:
        user_list.append(k)
        
print 'origin:', len(sample_dict)
print 'new:', len(new_pushes)
print 'with user:', len(user_list)


## Doing cross validation
from sklearn import cross_validation
dict_user = {} #training
dict_user_ans = {} #testing
for uid in user_list:
    training_set = sample_dict_jieba[uid]
    cv = cross_validation.KFold(len(training_set), n_folds=3, indices=True, shuffle=False, random_state=None, k=None)
    
    for traincv, testcv in cv:    
        dict_user[uid] = []
        for i in traincv:
            dict_user[uid].append(training_set[i])
            
        dict_user_ans[uid] = []
        for i in testcv:
            dict_user_ans[uid].append(training_set[i])
            

## prepare for trainset and testset(ans)
import pttfunc
dict_user_count = pttfunc.count_dict(dict_user) #count the freq
dict_user_count_ans = pttfunc.count_dict(dict_user_ans) #count the freq for ans(test)

## count all using words freq, preparing for generating sorted sw_list 
count_all = Counter()
for v in dict_user_count.values():
    count_all += Counter(v)
from collections import OrderedDict
dict_all_count = OrderedDict(sorted(dict(count_all).items(), key=lambda t: t[1], reverse=True))

## set stopword percentage(in global push)
W_PERCENT = 0.025
word = dict_all_count.keys()
stopwords = [x for x in word if dict_all_count[x] >= dict_all_count[word[int(len(word) * W_PERCENT)]]]
#rarewords = [x for x in word if dict_all_count[x] <= dict_all_count[word[int(len(word) * (1-W_PERCENT))]]] #0.975

## stop word list
sw_list = [x for x in stopwords] 
## rare word list
#rw_list = [x for x in rarewords] 
print 'total word', len(word)
print 'stop word', len(sw_list)
#print 'rare word', len(rw_list)

## select stopword less than 6 word
K_NUMWORD = 6
sw_count_dict = {}
for w in sw_list:
    sw_count_dict.setdefault(len(w),[]).append(w)
selected_sw = []
for k in sw_count_dict.keys()[0:K_NUMWORD]:
    print 'num_word:',k,'\t',len(sw_count_dict[k])
    selected_sw += sw_count_dict[k]
print 'after filter:', len(selected_sw)


## generate training/testing vectors by sw_word freq / total wordfreq
general_vec = {}
for uid in user_list: # for each user id
    user_len = sum([len(x) for x in dict_user_count[uid]]) #total word freq
    if user_len > 0:
        vec = [dict_user_count[uid].get(w, 0) for w in selected_sw]
        g_vec = [float(x) / user_len for x in vec]
        general_vec[uid] = g_vec

general_vec_ans = {}
for uid in user_list: # for each user id
    user_len = sum([len(x) for x in dict_user_count_ans[uid]]) #total word freq
    #print sum(v.values())
    if user_len > 0:
        #vec = [jvc_grams_count[idx].get(w, 0) for w in new_sw_list] #stopword without function words

        vec = [dict_user_count_ans[uid].get(w, 0) for w in selected_sw]
        g_vec = [float(x) / user_len for x in vec]
        general_vec_ans[uid] = g_vec

##calculate similarity for each user pair

import pttfunc
import numpy as np
sim_list = np.array((0.0, 0.0, 0.0))
for i in xrange(len(user_list)):
    for j in xrange(len(user_list)):
        #wj_sw = pttfunc.weighted_jaccard(general_vec[user_list[i]], general_vec_ans[user_list[j]])
        
        sim_list = np.vstack((sim_list, np.array((round(float(i),1), round(float(j),1), wj_sw))))

#sorting
sim_list = sim_list[1:]
sim_list = sim_list[sim_list[:,2].argsort()]
sim_list = sim_list[::-1]

#make final list
user_sim_list = []
for i, j, sim in sim_list:
    user_sim_list.append((user_list[int(i)], user_list[int(j)] + 'ANS', sim))

## evaluation
a = 0
for i,j,sim in user_sim_list:
    if i == j[:-3] and sim > 0.3:
        a+=1
b = len(user_list)
print float(a)/ b


Building prefix dict from /Users/joekaojoekao/PycharmProjects/push/github/dict.txt.big ...
DEBUG:jieba:Building prefix dict from /Users/joekaojoekao/PycharmProjects/push/github/dict.txt.big ...
Loading model from cache /var/folders/qn/v8s1xx6d7qgclyhkk1t701zc0000gn/T/jieba.u28e5fd167ca6b25e789769ad04c48668.cache
DEBUG:jieba:Loading model from cache /var/folders/qn/v8s1xx6d7qgclyhkk1t701zc0000gn/T/jieba.u28e5fd167ca6b25e789769ad04c48668.cache
Loading model cost 1.19007706642 seconds.
DEBUG:jieba:Loading model cost 1.19007706642 seconds.
Prefix dict has been built succesfully.
DEBUG:jieba:Prefix dict has been built succesfully.


origin: 28
new: 28
with user: 19
total word 715
stop word 18
num_word: 1 	18
after filter: 18
0.210526315789


In [233]:
from IPython.html.widgets import interact
def evaluate(x):
    a = 0
    for i,j,sim in user_sim_list:
        if i in j and sim >= x:
            a+=1
    b = len(user_list)
    print float(a) / b
interact(evaluate, x=(0.0,1.0,0.00001));

0.0526315789474


In [239]:
for i,j in cv:
    print i,j
for k,v in dict_user.items():
    print k,v
    break
for k,v in dict_user_ans.items():
    print k,v
    break

[3 4 5 6 7] [0 1 2]
[0 1 2 6 7] [3 4 5]
[0 1 2 3 4 5] [6 7]
akaihuang [('\xe9\x80\x99\xe6\x9c\x89\xe9\xbb\x9e\xe5\x81\x87\xe5\x90\xa7\xef\xbc\x8c\xe7\x8f\xbe\xe5\x9c\xa8\xe6\x98\xaf\xe6\x89\x8b\xe6\xa9\x9f\xe6\xb2\x92\xe9\x9b\xbb\xe4\xba\x86\xe5\x97\x8e XD', [u'\u9019', u'\u6709\u9ede', u'\u5047', u'\u5427', u'\uff0c', u'\u73fe\u5728', u'\u662f', u'\u624b\u6a5f', u'\u6c92\u96fb', u'\u4e86', u'\u55ce', u' ', u'XD']), ('\xe4\xbd\xa0\xe9\x82\x8f\xe8\xbc\xaf\xe6\x9c\x89\xe5\x95\x8f\xe9\xa1\x8c...\xef\xbc\x8c\xe5\xa4\xa7\xe9\x99\xb8\xe4\xb8\x8d\xe6\x89\xbf\xe8\xaa\x8d\xe9\x9b\x99\xe9\x87\x8d\xe3\x80\x90\xe5\x9c\x8b\xe3\x80\x91\xe7\xb1\x8d\xe6\xb2\x92\xe9\x8c\xaf\xef\xbc\x8c\xe4\xb8\x8d\xe9\x81\x8e...', [u'\u4f60', u'\u908f\u8f2f', u'\u6709', u'\u554f\u984c', u'...', u'\uff0c', u'\u5927\u9678', u'\u4e0d', u'\u627f\u8a8d', u'\u96d9\u91cd', u'\u3010', u'\u570b', u'\u3011', u'\u7c4d\u6c92', u'\u932f', u'\uff0c', u'\u4e0d\u904e', u'...']), ('\xe4\xb8\xad\xe8\x8f\xaf\xe6\xb0\x91\xe5\x9c\x8b\xe5\x

In [49]:
##import processed push data

#/Users/joekaojoekao/PycharmProjects/push/github/visualized/select_pushes1000_1.txt
with open('/Users/joekaojoekao/PycharmProjects/push/github/visualized/select_pushes101.txt', 'rb') as fin:
    s = fin.read().split('\n')
    lines = []
    for line in s:
        lines.append(line.split('\t'))
    fin.close()

sample_dict = {}
for i in lines:
    
    #print len(i)
    #if len(i) >= 3:
    if len(i) != 2:
        continue
    uid = i[0]
    push = i[1]
    if uid not in sample_dict:
        sample_dict[uid] = [push]
    else:
        sample_dict[uid].append(push)
#print sample_dict


In [100]:
##segment pushes

import jieba
import jieba.analyse
jieba.set_dictionary('dict.txt.big')
jieba.analyse.set_stop_words('stopword_pool/merged_stopword.txt')

sample_dict_jieba = {}
for uid in sample_dict.keys():
    pushes = sample_dict[uid]
    if len(pushes) > 0:
        sample_dict_jieba[uid] = []
        for push in pushes:
            seg_list = list(jieba.cut(push, cut_all=False))
            sample_dict_jieba[uid].append((push,seg_list))
#從ptt push開始處理
#我要uid:push&trunked word
#幹 要現切 因為產生的東西只剩uid+push(無順序無法查)


Building prefix dict from /Users/joekaojoekao/PycharmProjects/push/github/dict.txt.big ...
DEBUG:jieba:Building prefix dict from /Users/joekaojoekao/PycharmProjects/push/github/dict.txt.big ...
Dumping model to file cache /var/folders/qn/v8s1xx6d7qgclyhkk1t701zc0000gn/T/jieba.u28e5fd167ca6b25e789769ad04c48668.cache
DEBUG:jieba:Dumping model to file cache /var/folders/qn/v8s1xx6d7qgclyhkk1t701zc0000gn/T/jieba.u28e5fd167ca6b25e789769ad04c48668.cache
Loading model cost 4.15047812462 seconds.
DEBUG:jieba:Loading model cost 4.15047812462 seconds.
Prefix dict has been built succesfully.
DEBUG:jieba:Prefix dict has been built succesfully.


In [113]:
print sample_dict_jieba.values()[0]

[('\xe9\x80\x99\xe6\x9c\x89\xe9\xbb\x9e\xe5\x81\x87\xe5\x90\xa7\xef\xbc\x8c\xe7\x8f\xbe\xe5\x9c\xa8\xe6\x98\xaf\xe6\x89\x8b\xe6\xa9\x9f\xe6\xb2\x92\xe9\x9b\xbb\xe4\xba\x86\xe5\x97\x8e XD', [u'\u9019', u'\u6709\u9ede', u'\u5047', u'\u5427', u'\uff0c', u'\u73fe\u5728', u'\u662f', u'\u624b\u6a5f', u'\u6c92\u96fb', u'\u4e86', u'\u55ce', u' ', u'XD']), ('\xe4\xbd\xa0\xe9\x82\x8f\xe8\xbc\xaf\xe6\x9c\x89\xe5\x95\x8f\xe9\xa1\x8c...\xef\xbc\x8c\xe5\xa4\xa7\xe9\x99\xb8\xe4\xb8\x8d\xe6\x89\xbf\xe8\xaa\x8d\xe9\x9b\x99\xe9\x87\x8d\xe3\x80\x90\xe5\x9c\x8b\xe3\x80\x91\xe7\xb1\x8d\xe6\xb2\x92\xe9\x8c\xaf\xef\xbc\x8c\xe4\xb8\x8d\xe9\x81\x8e...', [u'\u4f60', u'\u908f\u8f2f', u'\u6709', u'\u554f\u984c', u'...', u'\uff0c', u'\u5927\u9678', u'\u4e0d', u'\u627f\u8a8d', u'\u96d9\u91cd', u'\u3010', u'\u570b', u'\u3011', u'\u7c4d\u6c92', u'\u932f', u'\uff0c', u'\u4e0d\u904e', u'...']), ('\xe4\xb8\xad\xe8\x8f\xaf\xe6\xb0\x91\xe5\x9c\x8b\xe5\x9c\x8b\xe7\xb1\x8d\xe5\x9c\xa8\xe4\xbb\x96\xe5\x80\x91\xe7\x9c\xbc\xe4

In [127]:
for p in sample_dict_jieba.values()[0]:
    print p[0]

這有點假吧，現在是手機沒電了嗎 XD
你邏輯有問題...，大陸不承認雙重【國】籍沒錯，不過...
中華民國國籍在他們眼中是【國】籍嗎 科科
大陸人在我方法律本來就不算外國人，這算基本常識了


In [ ]:
## 分成兩個question ans (cv結束後)再統計? 或是到generate vec的時候統計好了

In [51]:
#print ptt_pushes_freq_byID.values()[0]

#WRONG!
for k,v in ptt_pushes_freq_byID_new.iteritems():
    print k
    for kk,vv in v.iteritems():
        print kk,vv

NameError: name 'ptt_pushes_freq_byID_new' is not defined

In [145]:
import numpy as np
import json
#set gt_numword (meaningful), gt_numpush (active)
NUM_WORD_GT = 4
NUM_PUSH_GT = 3 #for 6 fold-cv

new_pushes = {}
for k in sample_dict.keys():
    gt_push = [x for x in sample_dict[k] if len(x) > NUM_WORD_GT]
    if len(gt_push) > 0:
        new_pushes[k] = gt_push

user_list = []
for k in new_pushes.keys():
    if len(new_pushes[k]) >= NUM_PUSH_GT:
        user_list.append(k)
        
print len(sample_dict)
print len(new_pushes)
print len(user_list)
print user_list

28
28
19
['akaihuang', 'jennywalk', 'C4891', 'morgan168', 'AAAB', 'AAAD', 'fst1985', 'aoiaoi', 'andy80209', 'fhuocrkt', 'wxtab019', 'keithking', 'wilsonno1', 'douglasyeh', 'Meow0129', 'chind', 'motorolla', 'Yakyuboy51', 'feather9298']


In [217]:
#import nltk
from sklearn import cross_validation
#training_set = nltk.classify.apply_features(extract_features, documents)
#dict_user = {} #training
#dict_user_ans = {} #testing
CV_TIMES = 3

#list_user = [{}] * 3
#list_user_ans = [{}] * 3

list_user = [{} for i in range(3)] 
list_user_ans = [{} for i in range(3)] 


for uid in user_list:
    training_set = sample_dict_jieba[uid]
    #cv = cross_validation.KFold(len(training_set), n_folds=3, indices=True, shuffle=False, random_state=None, k=None)
    #trainset = [0,1,2,3,4,5,6,7,8,9]
    cv = cross_validation.ShuffleSplit(len(training_set), n_iter=CV_TIMES, test_size=0.33) # 2/3 for train, 1/3 for test, do 3 times  
    
    #for i in training_set:
        #print i[0]
    
    for n, (traincv, testcv) in enumerate(cv): #每個user都有3組cv的推文
        #print n, traincv, testcv
        
        train_list = [training_set[i] for i in traincv]
        test_list = [training_set[i] for i in testcv]
        
        for i in zip(*train_list)[0]:
            print i
        print '---'
        for i in zip(*test_list)[0]:
            print i
        
        list_user[n][uid] = train_list
        list_user_ans[n][uid] = test_list
        
#         list_user[n][uid] = []
#         for i in traincv:
#             list_user[n][uid].append(training_set[i]) #list_user[第n次cv][user uid]
        
#         list_user_ans[n][uid] = []
#         for i in testcv:
#             list_user_ans[n][uid].append(training_set[i])
        
        #print uid
        #for v in list_user[0][uid]:
            #print v[0]
            #for vv in v:
                #print vv[0]


# for p in list_user[0].values()[1]:
#     for pp in p:
#         print p[0]
# print '-----------'
# for p in list_user[1].values()[1]:
#     for pp in p:
#         print p[0]

In [243]:
for k, v in list_user[0].iteritems():
    print k
    for vv in v:
        print vv[0]

akaihuang
中華民國國籍在他們眼中是【國】籍嗎 科科
大陸人在我方法律本來就不算外國人，這算基本常識了
feather9298
預算再追加五百億
我今天睡過頭只能領便當
推我應該有五百內吧
果然壓力大XDDD
可望=不可能
jennywalk
全家，作超商可要搶破頭呢！
私底下也能解決，把阿桑罵罵就好，現除了道歉火人外沒法
昨晚就有建議原po了，丟上網路語論下，會較嚴重。
這樣變鬧版-.-？
了，我個人是認為罵罵，東西當下拿回來就好。
沒跟到，不過請問一下反對的理由是什麼？新計程還遠通？
wilsonno1
中立 跟中天大概是差不多的意思
姚立明真的好強orz
chind
哦? 鶴拳的影片 能附上網址嗎? 此鶴拳跟洪門拳的鶴拳不一樣
能免則免 你頂多把對方打倒 對方卻很有可能把你砍死
真正危險的地方是在 被動攻擊的狀態 不排除有偷襲的可能
網頁版的藍色比較亮 可以參考 這藍色在BBS是最亮了
C4891
明明就是綠黨在背後煽動的！又是藍綠惡鬥！逢中必反！！！
翻了一大堆稅金買的警車誰要賠啊 初估至少損失一億人民幣！
原po是台灣人吧 香港人是指那照片的拍攝者
沒禮貌的暴民、恐怖份子！
某樓的不要造謠 哪有死人啊 明明只有「拍拍肩膀」！！！
以前台灣人抗爭也滿強捍的....
douglasyeh
014之亂要亂個014年
35秒13k比較有可能
罵得很好啊
一樓也危險了...
morgan168
來來來來來來來來來來來來來來來來來來來來來來來來
大家辛苦了! 真的很感動! 希望真相早日水落石出!
wxtab019
記得備份讓他等著上法院了
這個有說 好像那時候學校公告還沒有貼報名連結
還是自己找出來然後再FB傳的
如果被告就說新聞說這樣都可以 而且還無罪了
反正還會噴水消暑
到中華電信回報好了...
自以為這種活動很好玩?P幣有用? 之後大家都換用人民幣了
哪來的山豬...我都沒看過
我走了 ~~我又回來了~~~我又走了 ~~~我又回來了~~~
不是柔性勸離 拍拍肩膀而已嗎?一定是哪裡搞錯了
去掛急診不要看門診阿
反正多弄幾個坑出來坦 全部弄在一起就有些可以趁亂摸過
有人送保險套 有人送素肚 剛剛好阿
這不是看條文內容就知道了?
mod好像沒有可以回報讓他下架的地方?
某版昨天一堆高潮的現在又對這個消息當作完全沒看到了
以後走在大馬路上的人 如果被性騷擾也都自

In [244]:
for k, v in list_user_ans[0].iteritems():
    print k
    for vv in v:
        print vv[0]

akaihuang
你邏輯有問題...，大陸不承認雙重【國】籍沒錯，不過...
這有點假吧，現在是手機沒電了嗎 XD
feather9298
不說謊才是新聞
高調!!!!!
wooooow
jennywalk
2000美給一個外國人喇舌台灣女，該關心的不關心
這種待遇在高雄屏東真的差不多阿。
記者加油！！
樓上有人說不如作超商？我屏東老家，全村也才二小七一
wilsonno1
那請趕快叫中國去宣布新加坡是中國不可分割的一部分
chind
除非平常很有sense 不讓別人靠近一公尺以內
避免晚上獨自一個人走暗巷 少跟人結仇 財不露白 這些較重要
我猜對了 果然是高老師
C4891
大家等著看這群暴民散場後地上垃圾有沒有撿乾淨！！！
韓國人平常也滿奴的啊 只是抗爭的時候才會強捍
-----------------鄉民要團結，團結真有力-----------------
douglasyeh
肯德基分店這麼少 結果光大里就開2家
超級菜市場名耶哈哈
5566還是去NBA板反串吧
morgan168
硍! 晚上要作惡夢惹~
看來高雄(譚)市需要蝙蝠俠!!
wxtab019
有些站好像過路的 可以不用買票進去
警察和政府官員不是都說警察沒打人?
還好我家沒裝第四台 不用看這種節目...
我走在你身邊會忍不住 這樣是不是也不會被告?改天去試試
所以如果他在自己女兒旁邊 他也睡不下去嗎?
應該可以去NCC檢舉了...只是不知道會不會被吃掉就是...
不過也是4/13 就有人發現 然後中正就已經有在FB上傳了
自己做錯事還要別人求你向他道歉? 要不要跪下來求你?
先想想造成抗爭的原因是什麼吧 如果大家都過得很好
停工一下 工人回去剛好特休修一修 過幾天繼續開工啦
公共場所就大家都可以評論?走在路上不是也可以說小姐好辣
有人會無聊沒事跑去抗爭嗎
keithking
我猜明天就是立委帶兒子出來，然後兒子哭哭道歉崩潰這樣
=============以下開放記者入場，GO!==================
幫高調 打到SAH是怎樣… 往死裡打？
記者快來抄
AAAB
文章有po在wanted但可能是很難徵求, 沒有人回應...
插座是為了想用麥克風呼籲過路民眾晚上6點參加凱道的活動
Meow0129
中小企業主很多都自己兼員工幹上來的 會寫PHP很奇怪?
中小企業主很多都自己兼員工幹上

In [209]:
list_user[0].update = dict('幹你嗎的真的假的')
print list_user[0]['akaihuang'][0][0]

TypeError: 'tuple' object does not support item assignment

In [149]:
for l in list_user:
    print l
#a = list_user[1]['akaihuang']
#for p in a:
#    print p[0]

# print len(dict_user.values()[0])
# print len(dict_user.values()[1])
# print len(dict_user.values())
# a = dict_user.values()[1]

# for push in a[0]:
#     print push[0]



{'akaihuang': [('\xe5\xa4\xa7\xe9\x99\xb8\xe4\xba\xba\xe5\x9c\xa8\xe6\x88\x91\xe6\x96\xb9\xe6\xb3\x95\xe5\xbe\x8b\xe6\x9c\xac\xe4\xbe\x86\xe5\xb0\xb1\xe4\xb8\x8d\xe7\xae\x97\xe5\xa4\x96\xe5\x9c\x8b\xe4\xba\xba\xef\xbc\x8c\xe9\x80\x99\xe7\xae\x97\xe5\x9f\xba\xe6\x9c\xac\xe5\xb8\xb8\xe8\xad\x98\xe4\xba\x86', [u'\u5927\u9678', u'\u4eba', u'\u5728', u'\u6211\u65b9', u'\u6cd5\u5f8b', u'\u672c\u4f86', u'\u5c31', u'\u4e0d\u7b97', u'\u5916\u570b\u4eba', u'\uff0c', u'\u9019\u7b97', u'\u57fa\u672c\u5e38\u8b58', u'\u4e86']), ('\xe4\xbd\xa0\xe9\x82\x8f\xe8\xbc\xaf\xe6\x9c\x89\xe5\x95\x8f\xe9\xa1\x8c...\xef\xbc\x8c\xe5\xa4\xa7\xe9\x99\xb8\xe4\xb8\x8d\xe6\x89\xbf\xe8\xaa\x8d\xe9\x9b\x99\xe9\x87\x8d\xe3\x80\x90\xe5\x9c\x8b\xe3\x80\x91\xe7\xb1\x8d\xe6\xb2\x92\xe9\x8c\xaf\xef\xbc\x8c\xe4\xb8\x8d\xe9\x81\x8e...', [u'\u4f60', u'\u908f\u8f2f', u'\u6709', u'\u554f\u984c', u'...', u'\uff0c', u'\u5927\u9678', u'\u4e0d', u'\u627f\u8a8d', u'\u96d9\u91cd', u'\u3010', u'\u570b', u'\u3011', u'\u7c4d\u6c92', u'\u9

In [227]:
# real do cv 
final_list = []

for i in xrange(CV_TIMES):
    dict_user = list_user[i]
    dict_user_count = pttfunc.count_dict(dict_user) #count the freq
    
    dict_user_ans = list_user_ans[i]
    dict_user_count_ans = pttfunc.count_dict(dict_user_ans) #count the freq for ans(test)
    from collections import Counter
    count_all = Counter()
    for v in dict_user_count.values():
        count_all += Counter(v)

    from collections import OrderedDict
    dict_all_count = OrderedDict(sorted(dict(count_all).items(), key=lambda t: t[1], reverse=True))

    ##gen sw
    word = dict_all_count.keys()
    W_PERCENT = 0.025 #0.025
    ## setting for stopword & rareword percentage
    stopwords = [x for x in word if dict_all_count[x] >= dict_all_count[word[int(len(word) * W_PERCENT)]]]
    rarewords = [x for x in word if dict_all_count[x] <= dict_all_count[word[int(len(word) * (1-W_PERCENT))]]] #0.975

    ## stop word list
    sw_list = [x for x in stopwords] 
    ## rare word list
    rw_list = [x for x in rarewords] 
    print 'total word', len(word)
    print 'stop word', len(sw_list)
    print 'rare word', len(rw_list)


    sw_count_dict = {}
    for w in sw_list:
        sw_count_dict.setdefault(len(w),[]).append(w)


    K_NUMWORD = 6
    selected_sw = []
    for k in sw_count_dict.keys()[0:K_NUMWORD]:
        print 'num_word:',k,'\t',len(sw_count_dict[k])
        selected_sw += sw_count_dict[k]
    print len(selected_sw)
    
    
    
    ##gen vec
    import itertools
    general_vec = {}
    for uid in user_list: # for each user id
        user_len = sum([len(x) for x in dict_user_count[uid]]) #total word freq
        #print sum(v.values())
        if user_len > 0:
            #vec = [jvc_grams_count[idx].get(w, 0) for w in new_sw_list] #stopword without function words

            vec = [dict_user_count[uid].get(w, 0) for w in selected_sw]
            g_vec = [float(x) / user_len for x in vec]
            general_vec[uid] = g_vec


    general_vec_ans = {}
    for uid in user_list: # for each user id
        user_len = sum([len(x) for x in dict_user_count_ans[uid]]) #total word freq
        #print sum(v.values())
        if user_len > 0:
            #vec = [jvc_grams_count[idx].get(w, 0) for w in new_sw_list] #stopword without function words

            vec = [dict_user_count_ans[uid].get(w, 0) for w in selected_sw]
            g_vec = [float(x) / user_len for x in vec]
            general_vec_ans[uid] = g_vec

    print len(general_vec[user_list[0]])
    print len(general_vec_ans[user_list[0]])
    
    import pttfunc
    #comb = gb.getCombination(len(user_list))

    ##calculate similarity for each user pair

    sim_list = np.array((0.0, 0.0, 0.0))

    # for i, j in comb:
    #     wj_sw = pttfunc.weighted_jaccard(general_vec[user_list[i]], general_vec_ans[user_list[j]])
    #     #sim_list = np.append(sim_list, np.array((k1, k2, wj_sw), dtype=mtype))
    #     sim_list = np.vstack((sim_list, np.array((round(float(i),1), round(float(j),1), wj_sw))))

    for i in xrange(len(user_list)):
        for j in xrange(len(user_list)):
            wj_sw = pttfunc.weighted_jaccard(general_vec[user_list[i]], general_vec_ans[user_list[j]])
            sim_list = np.vstack((sim_list, np.array((round(float(i),1), round(float(j),1), wj_sw))))



    sim_list = sim_list[1:]
    sim_list = sim_list[sim_list[:,2].argsort()]
    sim_list = sim_list[::-1]



    user_sim_list = []
    for i, j, sim in sim_list:
        user_sim_list.append((user_list[int(i)], user_list[int(j)] + 'ANS', sim))
    
    final_list.append(user_sim_list)


0.0


In [241]:
from IPython.html.widgets import interact
def evaluate(x, y):
    a = 0
    user_sim_list = final_list[y]
    for i,j,sim in user_sim_list:
        if i in j and sim >= x:
            a+=1
    b = len(user_list)
    print float(a) / b


interact(evaluate, x=(0.0,1.0,0.01), y=(0, CV_TIMES-1));

0.368421052632


In [236]:
from IPython.html.widgets import *
import math
from matplotlib import pyplot as plt
def pltsin(f, a):
    print f*a
interact(pltsin, f=(1,10,0.1), a=(1,10,1));

27.0


In [72]:
import pttfunc
#print dict_user.values()[0]
for v in dict_user.values()[0]:
    print v
dict_user_count = pttfunc.count_dict(dict_user) #count the freq
dict_user_count_ans = pttfunc.count_dict(dict_user_ans) #count the freq for ans(test)
print '--'
print dict_user_count.values()[0]
for k,v in dict_user_count.values()[0].iteritems():
    print k,v

[('\xe4\xbd\xa0\xe9\x82\x8f\xe8\xbc\xaf\xe6\x9c\x89\xe5\x95\x8f\xe9\xa1\x8c...\xef\xbc\x8c\xe5\xa4\xa7\xe9\x99\xb8\xe4\xb8\x8d\xe6\x89\xbf\xe8\xaa\x8d\xe9\x9b\x99\xe9\x87\x8d\xe3\x80\x90\xe5\x9c\x8b\xe3\x80\x91\xe7\xb1\x8d\xe6\xb2\x92\xe9\x8c\xaf\xef\xbc\x8c\xe4\xb8\x8d\xe9\x81\x8e...', [u'\u4f60', u'\u908f\u8f2f', u'\u6709', u'\u554f\u984c', u'...', u'\uff0c', u'\u5927\u9678', u'\u4e0d', u'\u627f\u8a8d', u'\u96d9\u91cd', u'\u3010', u'\u570b', u'\u3011', u'\u7c4d\u6c92', u'\u932f', u'\uff0c', u'\u4e0d\u904e', u'...']), ('\xe9\x80\x99\xe6\x9c\x89\xe9\xbb\x9e\xe5\x81\x87\xe5\x90\xa7\xef\xbc\x8c\xe7\x8f\xbe\xe5\x9c\xa8\xe6\x98\xaf\xe6\x89\x8b\xe6\xa9\x9f\xe6\xb2\x92\xe9\x9b\xbb\xe4\xba\x86\xe5\x97\x8e XD', [u'\u9019', u'\u6709\u9ede', u'\u5047', u'\u5427', u'\uff0c', u'\u73fe\u5728', u'\u662f', u'\u624b\u6a5f', u'\u6c92\u96fb', u'\u4e86', u'\u55ce', u' ', u'XD'])]
[('\xe4\xbd\xa0\xe9\x82\x8f\xe8\xbc\xaf\xe6\x9c\x89\xe5\x95\x8f\xe9\xa1\x8c...\xef\xbc\x8c\xe5\xa4\xa7\xe9\x99\xb8\xe4\xb8\x8d

TypeError: unhashable type: 'list'

In [149]:
## count all using words freq, preparing for generating sw_list 
count_all = Counter()
for v in dict_user_count.values():
    count_all += Counter(v)

from collections import OrderedDict
dict_all_count = OrderedDict(sorted(dict(count_all).items(), key=lambda t: t[1], reverse=True))

In [150]:
i = 0
for k,v in dict_all_count.iteritems():
    i+=1
    if i == 10:
        break
    print k,v

  7842
的 2076
! 2066
了 1246
， 1054
是 947
~ 877
? 860
= 849


In [151]:
##set threshold to generate stop words
# with open('/Users/joekaojoekao/PycharmProjects/push/github/gossiping_push_jieba_freq_all_sorted_json.txt', 'rb') as fout:
#     ptt_pushes_freq_all_sorted = json.load(fout)
#     fout.close()
# word = ptt_pushes_freq_all_sorted.keys()

word = dict_all_count.keys()
W_PERCENT = 0.025 #0.025
## setting for stopword & rareword percentage
stopwords = [x for x in word if dict_all_count[x] >= dict_all_count[word[int(len(word) * W_PERCENT)]]]
rarewords = [x for x in word if dict_all_count[x] <= dict_all_count[word[int(len(word) * (1-W_PERCENT))]]] #0.975

## stop word list
sw_list = [x for x in stopwords] 
## rare word list
rw_list = [x for x in rarewords] 
print 'total word', len(word)
print 'stop word', len(sw_list)
print 'rare word', len(rw_list)


sw_count_dict = {}
for w in sw_list:
    sw_count_dict.setdefault(len(w),[]).append(w)


K_NUMWORD = 6
selected_sw = []
for k in sw_count_dict.keys()[0:K_NUMWORD]:
    print 'num_word:',k,'\t',len(sw_count_dict[k])
    selected_sw += sw_count_dict[k]
print len(selected_sw)


total word 11827
stop word 302
rare word 7195
num_word: 1 	152
num_word: 2 	139
num_word: 3 	8
num_word: 4 	2
num_word: 5 	1
302


In [200]:
print selected_sw[0:10]
print len(dict_user_count)

print sum([len(x) for x in dict_user_count[user_list[0]]])
for k,v in dict_user_count[user_list[0]].iteritems():
    print k,v
#print dict_user[user_list[0]]
print '================='
for k,v in dict_user_count_ans[user_list[200]].iteritems():
    print k,v

[u' ', u'\u7684', u'!', u'\u4e86', u'\uff0c', u'\u662f', u'~', u'?', u'=', u'\u6211']
718
44
要 1
崇拜 1
屌 1
他們 1
打 1
喔 2
Push 1
可怕 1
真的 1
堅持下去 1
! 2
  2
一定 1
不要 3
難過 1
菁英 1
XDDDDDDDD 1
靠北 1
好 3
灶神 1
！ 1
休想 1
的 3
了 1
謝謝 1
很 1
變 1
小 1
他們 1
太扯 1
這 1
可怕 1
真的 1
! 5
  3
重要 1
感人 1
這樣 1
馬 1
就 1
高調 3
政商 1
為 2
非常 1
好好 1
這篇 1
鋪路 1
生命 1
扯 1
所欲 1
不要臉 1
有錢 1


In [172]:
import itertools
general_vec = {}
for uid in user_list: # for each user id
    user_len = sum([len(x) for x in dict_user_count[uid]]) #total word freq
    #print sum(v.values())
    if user_len > 0:
        #vec = [jvc_grams_count[idx].get(w, 0) for w in new_sw_list] #stopword without function words
        
        vec = [dict_user_count[uid].get(w, 0) for w in selected_sw]
        g_vec = [float(x) / user_len for x in vec]
        general_vec[uid] = g_vec
        

general_vec_ans = {}
for uid in user_list: # for each user id
    user_len = sum([len(x) for x in dict_user_count_ans[uid]]) #total word freq
    #print sum(v.values())
    if user_len > 0:
        #vec = [jvc_grams_count[idx].get(w, 0) for w in new_sw_list] #stopword without function words
        
        vec = [dict_user_count_ans[uid].get(w, 0) for w in selected_sw]
        g_vec = [float(x) / user_len for x in vec]
        general_vec_ans[uid] = g_vec

print len(general_vec[user_list[0]])
print len(general_vec_ans[user_list[0]])

302
302


In [201]:
#play with all comb

#import combination as gb
import pttfunc
#comb = gb.getCombination(len(user_list))

##calculate similarity for each user pair

sim_list = np.array((0.0, 0.0, 0.0))

# for i, j in comb:
#     wj_sw = pttfunc.weighted_jaccard(general_vec[user_list[i]], general_vec_ans[user_list[j]])
#     #sim_list = np.append(sim_list, np.array((k1, k2, wj_sw), dtype=mtype))
#     sim_list = np.vstack((sim_list, np.array((round(float(i),1), round(float(j),1), wj_sw))))

for i in xrange(len(user_list)):
    for j in xrange(len(user_list)):
        wj_sw = pttfunc.weighted_jaccard(general_vec[user_list[i]], general_vec_ans[user_list[j]])
        sim_list = np.vstack((sim_list, np.array((round(float(i),1), round(float(j),1), wj_sw))))



sim_list = sim_list[1:]
sim_list = sim_list[sim_list[:,2].argsort()]
sim_list = sim_list[::-1]



user_sim_list = []
for i, j, sim in sim_list:
    user_sim_list.append((user_list[int(i)], user_list[int(j)] + 'ANS', sim))

In [218]:
print len([x for x in user_sim_list if x[0] in x[1] and x[0] != x[1]])

740


In [222]:
#evaluating (using interact later?)
from IPython.html.widgets import interact
def evaluate(x):
    a = 0
    for i,j,sim in user_sim_list:
        if i in j and sim >= x:
            a+=1
    b = len(user_list)
    print float(a) / b
interact(evaluate, x=(0.0,1.0,0.01));

NameError: global name 'user_sim_list' is not defined

In [186]:
#play with same(uncover ans)

#import combination as gb
import pttfunc
#comb = gb.getCombination(len(user_list))

##calculate similarity for each user pair

sim_list = np.array((0.0))
for i in xrange(len(user_list)):
    wj_sw = pttfunc.weighted_jaccard(general_vec[user_list[i]], general_vec_ans[user_list[i]])
    #sim_list = np.append(sim_list, np.array((k1, k2, wj_sw), dtype=mtype))
    sim_list = np.vstack((sim_list, np.array(wj_sw))) ##here without np.array can save more
    #sim_list = np.vstack((sim_list, np.array((round(float(idx1),1), round(float(idx2),1), wj_sw))))


##generate user pushes and sim file



sim_list = sim_list[1:]
sim_list = sim_list[sim_list[:,0].argsort()]
sim_list = sim_list[::-1]


user_sim_list = []
for i, sim in enumerate(sim_list):
    user_sim_list.append((user_list[i], sim[0]))


TypeError: 'int' object is not iterable

In [184]:
print user_sim_list[0:100]

[('heavenkghs', 0.72270742358078588), ('nysky', 0.65044247787610621), ('quiet113', 0.51063829787234039), ('john2557', 0.50720774633471588), ('duo0518', 0.5), ('tienhun', 0.5), ('joyjack', 0.5), ('vm9487', 0.5), ('aftersilence', 0.5), ('lingon', 0.5), ('toshizo', 0.5), ('yuminlin', 0.5), ('fhuocrkt', 0.5), ('theropod', 0.5), ('kkchen', 0.5), ('emptie', 0.5), ('h3178378', 0.49999999999999989), ('zxc12385', 0.4900822447992258), ('Amelie27', 0.47565871596774717), ('buddygirl', 0.4705882352941177), ('chiang017', 0.45644059014632715), ('fromwilda', 0.45591535663383564), ('carters', 0.45086011051000141), ('Magicwind', 0.44101074404853841), ('glory5566', 0.43087960802613168), ('feather9298', 0.43010752688172044), ('fjdkqp', 0.42379171957547646), ('cojinyu', 0.42124542124542175), ('gayya152535', 0.41593695271453596), ('hnrywang', 0.41342385333136589), ('bugya', 0.40913233452860631), ('JeanSijhih', 0.40572795195451883), ('joyjcc', 0.40330088870080416), ('jeffl0402', 0.39795177154614764), ('Valer

In [185]:



#sel_sim = [(x[0], x[1], x[2]) for x in sim_list if x[2] >= AVG_3SD]
#sel_sim = [(x[0], x[1], x[2]) for x in sim_list]
with open('/Users/joekaojoekao/PycharmProjects/push/github/cv_result/1000_1.txt', 'wb') as fout1:
    for sim in user_sim_list:
        line = sim[0] + ',' + str(sim[1]) +'\n'
        fout1.write(line.encode('utf-8'))
fout1.close()

# with open('/Users/joekaojoekao/PycharmProjects/push/github/visualized/select_pushes1000_' + str(n+1) + '.txt', 'wb') as fout2:
#     for k in sel_user:
#         for v in new_pushes[k]:
#             if k == '':
#                 continue
#             line = k + '\t' + v +'\n'
#             fout2.write(line.encode('utf-8'))
# fout2.close()

# print n